## Assignment Week 4

This week we are going to write a program that guesses passwords.

Two files are provided: 
  * ```words.txt``` - this file contains a list of English words, one word per line
  * ```passwordfile.txt``` - this file contais a list of username:passwordhash combinations, one per line. 



### The password file

A well-known way to authenticate users is through the use of a username and password. If the user can provide the correct password, the user is granted access. 

To check the password, the system uses a *password file*, in which known username, password (both strings) combinations are stored, one combination per line. However, for security reasons, passwords are not stored in plain-text, but an encrypted version (called a *hash*) of the password string is stored instead. Hashes are computed using the function ```encrypt_password()```, which is provided below. This way, if the password file is stolen, the passwords are not immediately visible.

For example, if username "John" has password "secret", then instead of:

```
John:secret
```
the following is stored in the password file (see also below):
```
John:2bb80d537b1da3e38bd30361aa855686bde0eacd7162fef6a25fe97bf527a25b
```

When John wants to login into the system, then John provides his username and plaintext password. Then the system uses the ```encrypt_password()``` function to compute the hash of the provided password and compares the result to what it finds in the password file for the user John. If both strings match each other, John is granted access, otherwise not. 


In [9]:
##
## The encrypt_password function.
## You can use this function in your answer.
##

import hashlib

def encrypt_password( passwd ):
    """Encrypt a plaintext password (a string). It returns the result.
    This encryption is one-way only, meaning it is not easy (impossible) to decrypt
    the encrypted password to find out the original plaintext password again."""
    return hashlib.sha256( passwd.encode() ).hexdigest()

In [2]:
## Some examples of using the encrypt_password function
encrypt_password("secret")

'2bb80d537b1da3e38bd30361aa855686bde0eacd7162fef6a25fe97bf527a25b'

In [3]:
encrypt_password("assignment")

'ce7a7c10b0dfd96808cca64c88cf5c5e13b7775283bdc924767887bfa32c8fa1'

In [4]:
encrypt_password("#@SS1GNMENT!")

'b46352779bececb48e5a8d31e58684e8da139a944715bd44f0b930f3ac46bbca'

### Hacking (finding) the passwords 

Unfortunately, some users use passwords that are simply existing words or simple variations on them. A simple variations on words are to:
  * replace each instance of the letters i and I (small and capital i) with the number 1 (one)
  * replace each instance of the letters a and A (small and capital a) with the character @
  * add a hash sign (#) at the beginning of the word
  * add an exclamation mark (!) at the end of the word
  * capitalize all letters
  
For example, a user might use the password "#@SS1GNMENT!" which is a variation on the word "assignment".

You are going to write a program that uses a file containing a long list of possible passwords to try to find the plaintext passwords of users given a password file with encrypted passwords.

*Hint*: Make sure to break up your code in smaller parts (functions) and to first think about the solution (use paper!) before you start coding!

*Hint*: You may assume that a user either uses a word unaltered from the words file as a password or uses *all* 5 ways to alter an existing word from the file simultaneously as a password (so either "assignment" or "#@SS1GNMENT!").

*Hint*: You can use the provided The *encrypt_password()* function in your answer.

**Assignment 4.1**

Write a function that takes as argument a string and returns a new string in which
the argument string has been altered using the 5 mentioned alterations. For example,
"assignment" should be changed into "#@SS1GNMENT!".

Hint: You migth want to use the <A HREF="https://docs.python.org/2/library/string.html#string.replace">string.replace()</A> method in your answer.

In [12]:
# Convert the words to capital then change the A to @ and I to 1
def change_string(words):
    s = words.upper()
    
    final = s.replace('A','@').replace('I','1')
            
    return '#'+final+'!'

In [6]:
change_string('assignment')

'#@SS1GNMENT!'

**Assignment 4.2**

Write a program that reads the file ```words.txt``` and that prompts a user for an encrypted (hashed) password and that finds and prints the plaintext password (if it can be found) of the encrypted password. For example:
```
Input encrypted password: b46352779bececb48e5a8d31e58684e8da139a944715bd44f0b930f3ac46bbca
Plaintext password is: #@SS1GNMENT!
```
Another example is:
```
Input encrypted password: ce7a7c10b0dfd96808cca64c88cf5c5e13b7775283bdc924767887bfa32c8fa1
Plaintext password is: assignment
```

**Note:** make sure to use a dictionary data-type in your solution

In [23]:
# Make a function that create dictionary from words.txt
a_file = open('words.txt','r')
    
def create_dict(file):
    a_dict = dict()
    for el in file.read().splitlines():
        a_dict[encrypt_password(el)] = el
        a_dict[encrypt_password(change_string(el))] = change_string(el)
        
    return a_dict

# Make a function that take 2 arguments such as encrypted password word and dictionary from 'words.txt'
def find_input(hashinput, a_dict):
    if hashinput in a_dict:
        print('Plaintext password is:', a_dict[hashinput])
    else:
        print('Encrypted password is not found')

# Main program to take the input and convert the hash256 password to plaintext password
hashinput = input('Input encrypted password: ')
dictionary_word = create_dict(a_file)
find_input(hashinput, dictionary_word)

Input encrypted password: b46352779bececb48e5a8d31e58684e8da139a944715bd44f0b930f3ac46bbca
Plaintext password is: #@SS1GNMENT!


**Assignment 4.3**

Write a program that reads the password file ```passwordfile.txt``` and the word list ```words.txt```  and for each line in the password file prints out the name and corresponding plaintext password, if you can find it (if not, print something appropriately).
For example, the output could be:
```
John   secret
Mary   #@SS1GNMENT!
Bob    Cannot find password, too complex!
Jane   python
Peter  #PYTHON!
...
```

In [30]:
# Make the dictionary of words file
# Hash 256 as key and original words as value

a_file = open('words.txt','r')
b_file = open('passwordfile.txt','r')

def words_dict(file_word):    
    a_dict = dict()    
    for el in file_word.read().splitlines():
        a_dict[encrypt_password(el)] = el
        a_dict[encrypt_password(change_string(el))] = change_string(el)
        
    return a_dict

# Make the dictionary of password file
# Hash256 as key and name as value

def password_dict(file_pass):
    b_dict = dict()    
    for item in file_pass.read().splitlines():
        b_dict[item.split(':')[1]] = item.split(':')[0]
        
    return b_dict

# Main program to find the password in words.txt database
def main_program(dict_word,dict_pass):
    
    for b_key, b_values in dict_pass.items():
        if b_key in dict_word:
            print(b_values, '\t', dict_word[b_key])
        else:
            print(b_values, '\t', 'Cannot find password, too complex!')

# calling function main program to execute
words_dict = words_dict(a_file)
pass_dict = password_dict(b_file)

main_program(words_dict,pass_dict)

John 	 #@@RDV@RK!
Mary 	 #FLEDGL1NG!
Bob 	 Cannot find password, too complex!
Jane 	 python
Peter 	 #PYTHON!
Julia 	 programmer
Mike 	 #UN1VERS1TY!
Alice 	 #ELECT1ONS!
Zach 	 zombies
Vicky 	 #TUTOR1@L!
